In [ ]:
# Library to upload files directly to Google Drive
!pip install gupload

     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
ERROR: earthengine-api 0.1.266 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.7.10 which is incompatible.
  Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
from sklearn.metrics import mean_squared_error, make_scorer
from xgboost import XGBRegressor
import xgboost
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import gc
import pickle
import sys

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Read Preprocessed data
id_dataset = '1Y_cNEgLNsATfL8BGb-Ovz4cpWqxy72JD'
data = drive.CreateFile({'id':id_dataset})   
data.GetContentFile('df_processed.pkl')

In [ ]:
df = pd.read_pickle('df_processed.pkl')

# Test Data

In [ ]:
#Note this trainset is equal to the X_train + X_val used to train the models
X_val = df[df["date_block_num"]==33].drop("target",axis=1)
X_test  = df[df["date_block_num"]==34].drop("target",axis=1)

y_val = df[df["date_block_num"]==33].target.clip(0,20)

In [ ]:
del df
gc.collect();

# Models

In [ ]:
#Read lgbm
id_model = '17OexlSrIG6wezpnNQjp-N9Cp9x7viTE4'
data = drive.CreateFile({'id':id_model})   
data.GetContentFile('lgbm.pkl')
lgbm = pickle.load(open('lgbm.pkl', 'rb'))

In [ ]:
#Read XGBoost
id_model = '1q4-PFr6sBzCN0rFSUq0gQqwvUScFWgpW'
data = drive.CreateFile({'id':id_model})   
data.GetContentFile('xgb.pkl')
xgb = pickle.load(open('xgb.pkl', 'rb'))

In [ ]:
#Get Train & Test Meta data
lgbm_preds_val = lgbm.predict(X_val).clip(0,20)
xgb_preds_val  = xgb.predict(X_val).clip(0,20)
lgbm_preds_test = lgbm.predict(X_test).clip(0,20)
xgb_preds_test  = xgb.predict(X_test).clip(0,20)

val_meta_data = np.column_stack([lgbm_preds_val,xgb_preds_val])
test_meta_data  = np.column_stack([lgbm_preds_test,xgb_preds_test])

In [ ]:
# test_meta_data_2 = np.max(test_meta_data,axis=1)

# Ensemble (Linear Regression)

As it was taught in 

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(val_meta_data, y_val)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
submit = pd.DataFrame(lr.predict(test_meta_data).clip(0,20))
submit.index.name = "ID"
submit.rename(columns={0:"item_cnt_month"},inplace=True)

In [ ]:
submit.to_csv("submit_stacked.csv")

LB Score: 0.90843